# Importing libraries

In [ ]:
import re
import sqlite3
import pandas as pd
import openai
import os
from dotenv import load_dotenv
from langchain_community.utilities import SQLDatabase
import gradio as gr 

# Create LLM instance

In [ ]:
## TODO: convert to dot env files
llm = openai.AzureOpenAI(
        azure_endpoint="https://oai-eus2-001.openai.azure.com/",
        api_key="70ac8a66915a49cfb6c833f694d7ed24",
        api_version="2023-05-15"
)

# Read data and connetc to database

In [ ]:
def read_csv_to_dataframe():
    """
    Reads a CSV file and converts it into a DataFrame.

    Args:
    - file_path (str): The file path of the CSV file to be read.

    Returns:
    - df (pd.DataFrame): The DataFrame containing the data from the CSV file.
    """
    # Read the CSV file into a DataFrame
    file_path = get_file_path()
    df = pd.read_csv(file_path)

    return df

def get_file_path():
    # Specify the file path of the CSV file
    file_path = 'updated_data.csv'
    return file_path
def connect_to_database(database_name):
    """
    Connect to the SQLite database.

    Parameters:
    - database_name: Name of the SQLite database.

    Returns:
    - Connection object and Cursor object.
    """
    conn = sqlite3.connect(database_name)
    cursor = conn.cursor()
    return conn, cursor

In [ ]:
def create_loan_data_table(cursor):
    """
    Create a loan_data table in the SQLite database.

    Parameters:
    - cursor: Cursor object for executing SQL queries.
    """
    create_table_query = '''CREATE TABLE IF NOT EXISTS loan_data (
                            loan_id INTEGER PRIMARY KEY,
                            name TEXT NOT NULL,
                            date_of_birth DATE NOT NULL,
                            no_of_dependents INTEGER NOT NULL,
                            education TEXT NOT NULL,
                            self_employed TEXT NOT NULL,
                            income_annum INTEGER NOT NULL,
                            loan_amount INTEGER NOT NULL,
                            loan_term INTEGER NOT NULL,
                            cibil_score INTEGER NOT NULL,
                            residential_assets_value INTEGER NOT NULL,
                            commercial_assets_value INTEGER NOT NULL,
                            luxury_assets_value INTEGER NOT NULL,
                            bank_asset_value INTEGER NOT NULL,
                            loan_status TEXT NOT NULL,
                            CIBIL_rating TEXT NOT NULL,
                            income_level TEXT NOT NULL,
                            loan_rating TEXT NOT NULL,
                            loan_term_type TEXT NOT NULL,
                            dependent_level TEXT NOT NULL,
                            edu_status INTEGER NOT NULL,
                            type_employment INTEGER NOT NULL,
                            loan_approval INTEGER NOT NULL
                        );
                    '''
    cursor.execute(create_table_query)

def insert_data_into_table(conn, cursor, dataframe, table_name):
    """
    Insert data from a DataFrame into the specified table in the SQLite database.

    Parameters:
    - conn: Connection object to the database.
    - cursor: Cursor object for executing SQL queries.
    - dataframe: Pandas DataFrame containing the data to insert.
    - table_name: Name of the table to insert data into.
    """
    dataframe.to_sql(table_name, conn, if_exists='append', index=False)

    # Commit changes and close connection
    conn.commit()
    conn.close()

def generate_sql_query(user_input):
    prompt = f"""

    ## Problem Statement:
    You are a knowledgeable and friendly SQLite expert assisting users with database queries. 
    Given an input question, create a syntactically correct SQLite query that adheres to specific guidelines. Gives max 5 outputs.
    ## Input Question:
    User: {user_input}

    ## Guidelines for Query Generation:
    1. If the user specifies a specific number of examples to obtain, limit the query results using the `LIMIT` clause.
    2. Use only the column names from the provided database schema (see below). Avoid querying non-existent columns.
    3. Only query the columns necessary to answer the question. If there's a slight difference in the column name mentioned in the user's question and the actual column name in the table (e.g., "cibil value" instead of "cibil_rating"), perform a similarity search to find the most relevant column for the query.
    4. Do query for all the columns if not specified or else query the relevant columns and the name column.

    ## Database Schema:
    Use the following table schema when creating your queries:
    ```sql
    CREATE TABLE loan_data (
    loan_id INT NOT NULL,
    name VARCHAR(255) NOT NULL,
    date_of_birth DATE NOT NULL,
    no_of_dependents INT NOT NULL,
    education VARCHAR(255) NOT NULL,
    self_employed VARCHAR(255) NOT NULL,
    income_annum INT NOT NULL,
    loan_amount INT NOT NULL,
    loan_term INT NOT NULL,
    cibil_score INT NOT NULL,
    residential_assets_value INT NOT NULL,
    commercial_assets_value INT NOT NULL,
    luxury_assets_value INT NOT NULL,
    bank_asset_value INT NOT NULL,
    loan_status VARCHAR(255) NOT NULL,
    CIBIL_rating VARCHAR(255) NOT NULL,
    income_level VARCHAR(255) NOT NULL,
    loan_rating VARCHAR(255) NOT NULL,
    loan_term_type VARCHAR(255) NOT NULL,
    dependent_level VARCHAR(255) NOT NULL,
    edu_status INT NOT NULL,
    type_employment INT NOT NULL,
    loan_approval INT NOT NULL,
    PRIMARY KEY (loan_id)
    );
    Response Format:
    Your response should strictly adhere to the following format:
    Example: 
    1. User: "What is the Cibil value of loan id 1?"
   Query Generated: "SELECT cibil_rating, name FROM loan_data WHERE loan_id = 1;"

    2. User: "Show me the assets values for loan id 2 and loan id 3."
   Query Generated: "SELECT residential_assets_value, commercial_assets_value, luxury_assets_value, name FROM loan_data WHERE loan_id IN (2, 3);"

    3. User: "Is the income level of John higher than the average?"
   Query Generated: "SELECT income_level, name FROM loan_data WHERE name = 'John';"

    4. User: "What are the education levels of all self-employed individuals?"
   Query Generated: "SELECT education, name FROM loan_data WHERE self_employed = 'Yes';"

    5. User: "Give me details of loan approval for loan id 4."
   Query Generated: "SELECT loan_approval, name FROM loan_data WHERE loan_id = 4;"
    Provide the generated SQL query only , without any additional characters or phrases.
    Avoid including any text other than the SQL query in the response.
    Limitations:
    For irrelevant questions or queries that cannot be translated into a SQL query, return a response: "Not able to generate query." only and no additonal characters.

    """
    response = llm.chat.completions.create(
    model="GPT35TURBO",
    temperature=0.0,
    max_tokens=1000,
    top_p=1,
    messages=[
        {"role": "user", "content": prompt}
    ],
    )
    sql_query = response.choices[0].message.content
    if sql_query=='' or sql_query is None or 'sql_query' =="Not able to generate query.":
        return "Sorry there is no data available in the database for your question. Please check your query again."
    sql_query_result = execute_sql_query(sql_query)
    return sql_query_result

def execute_sql_query(sql_query):
    # Connect to the database and execute the SQL query
    db = SQLDatabase.from_uri("sqlite:///credit_score_loan_data.db")
    result = db.run(sql_query)
    # print(result)
    return result

def process_query_result(result, user_query):
    prompt = f"""

    ## Problem Statement:
    You are a helpful assistant chatbot that interacts with a database to answer user queries. Given a sql query result {result} from the loan_data table, and the user query {user_query} , 
    generate a human-readable answer of the information in the sql query result, answering the information relevant to the user query {user_query}. 
    
    ## Input Data:
    The query result contains either or all of the following columns:
    - loan_id: Unique identifier for each loan
    - name: Name of the loan applicant
    - date_of_birth: Date of birth of the loan applicant
    - no_of_dependents: Number of dependents of the loan applicant
    - education: Education level of the loan applicant
    - self_employed: Employment status of the loan applicant
    - income_annum: Annual income of the loan applicant
    - loan_amount: Amount of loan applied for
    - loan_term: Term of the loan in months
    - cibil_score: Credit score of the loan applicant
    - residential_assets_value: Value of residential assets
    - commercial_assets_value: Value of commercial assets
    - luxury_assets_value: Value of luxury assets
    - bank_asset_value: Value of assets in bank
    - loan_status: Status of the loan application (approved or rejected)
    - CIBIL_rating: Credit rating of the loan applicant
    - income_level: Income level of the loan applicant
    - loan_rating: Rating of the loan application
    - loan_term_type: Type of loan term (short-term or long-term)
    - dependent_level: Level of dependence on loan
    - edu_status: Education status of the loan applicant
    - type_employment: Type of employment of the loan applicant
    - loan_approval: Approval status of the loan (1 for approved, 0 for rejected)
    The name column is always an input.
    ## Task:
    Based on the query result, create a natural language summary that provides meaningful insights into the loan data.
    Response format should be similar to:
    The loan for has been approved.
    ## Guidelines:
    - Ensure the summary is concise, informative, and easy to understand.
    - Include key statistics and trends from the query result.
    - Use natural language and avoid technical jargon.
    - Do not give the sql query result in the response you provide.
    - Include the name of the user from the sql query result in your response.
    - If loan is approved then say loan can be given and if loan is not approved then say loan cannot be given and give a reason why loan cannot be given.
    - If there is no sql query result or the sql query result is "Not able to generate query". Then give a generic response that you do not have the answer to the question.
    - If {result} has 1 in it for the loan approval column and the user_query was whether loan can be disbursed or not then say loan has been approved for the user with the user's name.
    """
    response = llm.chat.completions.create(
    model="GPT35TURBO",
    temperature=0.4,
    max_tokens=1000,
    top_p=0.92,
    messages=[
        {"role": "user", "content": prompt}
    ],
    )
    
    answer = response.choices[0].message.content
    return answer

def is_follow_up_question(user_query, chat_history):
 
    '''
    This function decides whether the question asked by a user is a follow up question to the previous question or not by passing it to LLM.
 
    '''
    context_list = [f"{qa[0]}\n{qa[1]}" for qa in chat_history if qa[0] is not None]
    if len(context_list)==0: #User is asking his first question of the conversation.
        return user_query
   
    else:
        context_prev_q_a= ('\n').join(context_list)
   
    prompt = f""""
 
   
    #Context:
    Your first task is to rephrase the current question (user query) into gramatically correct format. Post rephrasing, given a context list and a the rephrased current question, your task is to perform a semantic similarity to the previous question and determine if it is a follow-up to the previous questions or not.
    After this, you need to return a standalone version of the current question such that it can be answered individually.
    
    Output Format:
    Response: The model's response should be the standalone version of the follow-up question, or the current question if it is not a follow-up.
    
    Specifications:
    Task: Analyze the provided context, consisting of a previous question and its response, along with the current question, to determine if it is a follow-up question to the previous context.
    
    Instructions:
    1. Check if the user_query is grammatically and meaningfully correct. If it is not, then rephrase the user query such that it forms a grammatically correct question or statement.
    2. Determine if the current question is a follow-up to the previous conversation stored in the Context list.
    
    3. If the current question is a follow-up:
    - Generate a standalone version of the follow-up question based on the information from the previous question and its answer. Sometimes, the current question may be a follow-up question but it might not include Company Name,Year or Quarter. You must obtain the same information from the previous Question in the context list.
    - Return the standalone follow-up question.
    4. If the current question is not a follow-up, return the question as it is, treating it as a new question.
    5. The standalone question format must include identification and mention of Company, Quarter, and Year details.
    6. Avoid providing personal explanations for why a question is a follow-up; return the output directly.
    7. Ensure the standalone version of the follow-up question is clear and can be answered independently.
    8. The decision whether the current question is a follow-up or not has to be made very accurately. This is very important.
    
    Task Examples:
    
    Example 1:
    
    Current Question: "What is their cibil rating?"
    Decision to Rephrase: This question is grammatically correct so no need to rephrase.
    Previous Question Answer List: ["Question: What is the income level of loan id 1? \n Answer: The income level of Vikas with loan id 1 is high"]
    Decision: In the current question, 'Their' refers to Vikas with loan id 1. Hence the current question is a follow-up to the previous question and hence a standalone version of the current question must be generated.
    Response:
    What is the cibil rating of loan id 1?
    
    Example 2:

    Current Question: "What is the CIBIL rating for loan ID 2?"

    Decision to Rephrase: This question is grammatically correct so no need to rephrase.

    Previous Question Answer List: ["Question: What is the income level for loan ID 1? \n Answer: The income level for Vikas with loan ID 1 is high"]

    Decision: In the current question, the bank agent is asking a different question for a different user, about the CIBIL rating for another specific loan ID (loan ID 2). Hence it is a new question and not a follow-up question to the previous question.

    Response:
    What is the CIBIL rating for loan ID 2?


    Example 3:


    Current Question: "Can her loan be approved?"

    Decision to Rephrase: This question is grammatically correct so no need to rephrase.

    Previous Question Answer List:
    ["Question: What is the CIBIL rating for loan ID 2? \n Answer: The CIBIL rating of shalini with loan ID 2 is excellent",
    "Question: What is their loan term? \n Answer: The loan term of Shalini with loan id 2 is 8"]

    Decision: The current question is asking about loan approval where 'her' refers to shalini whose loan id is 2 it is a follow up question to a follow up question, which means it is a follow-up to the previous question and it's previous question. Hence, a standalone version of the current question must be generated.

    Response: Can loan be approved for loan ID 2?
    

    Example 4:

    Current Question: "What is the loan term for him?"

    Decision to Rephrase: This question is grammatically correct so no need to rephrase.

    Previous Question Answer List: ["Question: What is the loan approval status for loan ID 3? \n Answer: The loan approval status for John with loan ID 3 is Approved"]

    Decision: The bank agent is seeking information about the loan terms where "him" refers to John with loan Id 3 as seen in previous answer, which is a follow-up question related to the previous question about loan approval status. Hence, a standalone version of the current question must be generated.

    Response:
    What are the loan terms for loan ID 3?


    Response:
    Could you provide details about the loan approval status for loan ID 4?
    
    
    Additional Information:
    1. Rephrase the current question meaningfully and grammatically.
    2. Ensure the standalone question format includes identification and mention of Company, Quarter, and Year and the finanial information that is being asked for in the current or previous question.
    3. Avoid providing personal interpretations or explanations.
    4. Base the response solely on the given context without speculation.
    5. Adhere strictly to the specified output format and instructions.
    
    Restrictions:
    1. Avoid providing personal explanations or interpretations for why a question is a follow-up.
    2. Return the output directly without additional commentary. You need to return only the standalone question and nothing else.
    3. Adhere strictly to the provided specifications and instructions.
    
    Here is the Current Question: {user_query}
    Here is the Context for the Current Question which includes the previous question and answers: {context_prev_q_a}
    
    
    
    """
    response = llm.chat.completions.create(
    model="GPT35TURBO",
    temperature=0.6,
    max_tokens=5000,
    top_p=0.9,
    messages=[
          {"role" : "user", "content" : prompt}
    ],
    )
    generated_response = response.choices[0].message.content
    return generated_response

def get_multiple_question(user_query) -> list[str]:
    '''
    This function identifies whether a question is complex, and returns a list of simpler questions.
    '''
 
 
    prompt = f'''
    Context:
    Your task is to analyze the user's query and determine whether it can be split into simpler queries.
    Make a decision to see if the given User_query is complex or simple. If it is a complex question that can be split into sub-questions, please do.
    If the question is simple, let it be as it is.
 
    Specifications:
    Task: Analyze the user's query and determine whether it can be split into simpler queries.
 
    Instructions:
    1. If the user's query is complex, split it into subqueries and return only the subqueries.
    2. If the user's query is not complex, return the question as it is.
    3. Avoid providing explanations on how or why the question is being split.
    4. Return only the subqueries and nothing else.
 
    Task Examples:
 
    Example 1:
 
    Question: Compare the cibil rating of load id 1 and loan id 2.
    Decision: This query is asking the information of the cibil rating of two users with loan id 1 and loan id 2. Hence it is a complex question and it can be split into two simple question.
    Response:
    What is the cibil rating of loan id 1?
    What is the cibil rating of loan id 2?
 
    Example 2:
 
    Question: Compare the Cibil rating and Income level of Loan id 1 and loan id 2? Which user is better in terms of these two parameters?
    Decision: This query is asking the information of the Cibil rating and Income level of two users with load id 1 and loan id 2. Hence it is a complex question and it can be split into multiple simple questions.
   
    Response:
 
    What is Cibil rating of loan id 1?
    What is Income level of loand id 1?
    What is Cibil rating of loan id 2?
    What is Income level of loand id 2?
    Is cibil rating of loan id 1 greater than cibil rating of loan id 2?
    Is income level of loan id 1 greater than income level of loan id 2?
 
    Additional Information:
    1. Ensure that subqueries are generated accurately based on the complexity of the user's query.
    2. Avoid providing explanations or justifications for splitting the question.
    3. Base the response solely on the complexity of the user's query.
 
    Restrictions:
 
    1. Avoid providing explanations on how or why the question is being split.
    2. Strictly return only the simple sub-questions and nothing else.
 
    Here is the question to be split: {user_query}
   
'''
   
    response = llm.chat.completions.create(
    model="GPT35TURBO",
    messages=[
          {"role" : "user", "content" : prompt}
    ],
        temperature=0.0,
        max_tokens=4000,
        top_p=0.92,
    )
    generated_response = response.choices[0].message.content
 
    list_of_questions=re.split(r'[?.]', generated_response)
 
    for i,j in enumerate(list_of_questions):
        if j=='':
            del list_of_questions[i]
    punctuated_list = []
    for string in list_of_questions:
        string = string.replace("\n", "")
        if string.startswith(("What", "Where", "How", "Why", "When")):
            punctuated_list.append(string + "?")
        else:
            punctuated_list.append(string + ".")
           
 
    return punctuated_list

def generate_combined_summarized_response(all_the_llm_responses, user_query, punctuated_list):
    prompt = """
    Context:
        You are developing a chatbot that handles complex queries by splitting them into simpler questions and retrieving answers. 
        The chatbot has already processed the user's complex query and generated a list {all_the_llm_responses}, which contains all the answers to the sub-questions derived from the user query : {user_query}.

        Task:
        Your task is to generate a combined version of all the answers in {all_the_llm_responses}, taking into account the context from the original user query: {user_query}. 
        The goal is to create a cohesive response that addresses the user's complex query comprehensively.

        Instructions:
        1. Use the information from all_the_llm_responses to craft a unified response.
        2. Ensure that the response maintains coherence and relevance to the original user query.
        3. Incorporate context from the original user query as well as any relevant context from the list of simplified questions - {punctuated_list} if needed.
        4. The response should be clear, concise, and cover all aspects addressed in the complex query.
        5. Avoid duplicating information or providing conflicting responses.

        Task Examples:
        Example 1:
        User Query: "Compare the Cibil rating and Income level of Loan id 1 and loan id 2? Which user is better in terms of these two parameters?"
        List of Simplified Questions:
        - What is Cibil rating of loan id 1?
        - What is Income level of loan id 1?
        - What is Cibil rating of loan id 2?
        - What is Income level of loan id 2?
        - Is Cibil rating of loan id 1 greater than Cibil rating of loan id 2?
        - Is Income level of loan id 1 greater than Income level of loan id 2?

        all_the_llm_responses:
        - Cibil rating of loan id 1 is 750.
        - Income level of loan id 1 is High.
        - Cibil rating of loan id 2 is 800.
        - Income level of loan id 2 is Medium.
        - Cibil rating of loan id 1 is not greater than Cibil rating of loan id 2.
        - Income level of loan id 1 is greater than Income level of loan id 2.

        Combined Response:
        Based on the comparison between the Cibil ratings and Income levels of Loan id 1 and Loan id 2:
        - Cibil rating of loan id 1: 750 (High)
        - Cibil rating of loan id 2: 800 (Medium)
        - Income level of loan id 1: High
        - Income level of loan id 2: Medium
        Loan id 2 has a higher Cibil rating, but Loan id 1 has a higher income level, making it better in terms of these two parameters.

        Additional Information:
        1. Ensure that the response is logically structured and addresses all aspects of the complex query.
        2. Use the information from all_the_llm_responses effectively to construct the combined response.
        3. Maintain clarity and consistency throughout the response.


    """

    response = llm.chat.completions.create(
    model="GPT35TURBO",
    temperature=0.3,
    max_tokens=1000,
    top_p=0.92,
    messages=[
        {"role": "user", "content": prompt}
    ],
    )
    
    answer = response.choices[0].message.content
    return answer


def execute_if_multiple_queries(punctuated_list, user_query):
    if len(punctuated_list)>1: #TODO add this in outside function
        #then there are muliple questions
        all_the_llm_responses = []
        for query in punctuated_list:
            sql_query_result = generate_sql_query(query)
            # sql_query_result = execute_sql_query(sql_query)
            llm_response = process_query_result(sql_query_result,query)
            all_the_llm_responses.append(llm_response)
        response = generate_combined_summarized_response(all_the_llm_responses, user_query, punctuated_list)
    else: 
        sql_query_result = generate_sql_query(user_query)  #TODO check this
        # result = execute_sql_query(sql_query)
        response = process_query_result(sql_query_result,user_query)
    return response

        
def generate_credit_analysis(user_query, history):
    print(f"History is: {history}")
    if user_query.lower() in ["bye", "exit", "goodbye"]:
        return "Bye, Have a good day!"
    loan_data = read_csv_to_dataframe()
    # Connect to the database
    conn, cursor = connect_to_database('credit_score_loan_data.db')
    # Create the loan_data table
    create_loan_data_table(cursor)
    db = SQLDatabase.from_uri("sqlite:///credit_score_loan_data.db")
    greeting_list = ["hi", "hello", "hey", "helloo", "hellooo", "g morining", "gmorning", "good morning", "morning", "good day", "good afternoon", "good evening", "greetings", "greeting", "good to see you", "its good seeing you", "how are you", "how're you", "how are you doing", "how ya doin'", "how ya doin", "how is everything", "how is everything going", "how's everything going", "how is you", "how's you", "how are things", "how're things", "how is it going", "how's it going", "how's it goin'", "how's it goin", "how is life been treating you", "how's life been treating you", "how have you been", "how've you been", "what is up", "what's up", "what is cracking", "what's cracking", "what is good", "what's good", "what is happening", "what's happening", "what is new", "what's new", "what is neww", "g’day", "howdy"]
    # Sample loan data (replace this with your actual loan data DataFrame)
    if user_query.lower() in greeting_list:
        return "Hello there! I am here to help with answering the credit analysis questions of a user. How can I assist you today?"
    substrings_to_check = ['id', 'loan_id', 'loan number', 'loan_number', 'loan id']
    # Check if any substring is present in the query
    for substring in substrings_to_check:

        if substring in user_query.lower():
            # check if the question is a follow up
            standalone_question = is_follow_up_question(user_query, history)
            list_of_questions = get_multiple_question(standalone_question)
            response = execute_if_multiple_queries(list_of_questions, standalone_question)
            return response
        else:
            
            return "Please provide the loan id of the user."

 
